In [0]:
#Invocar a la librería que me da el formato del archivo
from pyspark.sql.types import *

#Ruta donde esta el archivo
path = "/FileStore/tables/data_test.txt"

#Definir los tipos de dato de cada campo
schema = StructType([\
                     StructField("ID",IntegerType(),True), \
                     StructField("ID2",IntegerType(),True), \
                     StructField("NIVEL_EDUCATIVO",StringType(),True), \
                     StructField("SEXO",StringType(),True), \
                     StructField("CATEGORIA_EMPLEO",StringType(),True), \
                     StructField("EXPERIENCIA_LABORAL",IntegerType(),True), \
                     StructField("ESTADO_CIVIL",StringType(),True), \
                     StructField("EDAD",IntegerType(),True), \
                     StructField("UTILIZACION_TARJETAS",IntegerType(),True), \
                     StructField("NUMERO_ENTIDADES",StringType(),True), \
                     StructField("DEFAULT",IntegerType(),True), \
                     StructField("NUM_ENT_W",FloatType(),True), \
                     StructField("EDUC_W",FloatType(),True), \
                     StructField("EXP_LAB_W",FloatType(),True), \
                     StructField("EDAD_W",FloatType(),True), \
                     StructField("UTIL_TC_W",FloatType(),True), \
                     StructField("PD",FloatType(),True), \
                     StructField("RPD",FloatType(),True), \
                  ])

df_test =spark.read.option("header",True).option("encoding", "ISO-8859-1").schema(schema).csv(path,sep=" ")

In [0]:
df_test_adjust = df_test.drop("ID2")
df_test.display()

In [0]:
from pyspark.sql import SQLContext

#Inicializar la instancia de SQL Context

#Al moomento de trabajar fuera de databricks es necesario inicializarlo (para el servicio de spark)
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [0]:
sqlContext.registerDataFrameAsTable(df_test_adjust,"df_test_adjust") # -> el segundo parametro solo es el nombre de la tabla

#Se registrará el df como una tabla para poder realizar consultas SQL, que viene instanciado de la clase sqlContext
#Luego de instanciar la clase de SQLContext, podemosregistrarlo como si fuera una tabla bd relacional



In [0]:
sqlContext.sql("select  count(*) from df_test_adjust").show()

In [0]:
sqlContext.sql("select * from df_test_adjust").show()

In [0]:
sqlContext.sql("select NIVEL_EDUCATIVO, count(*) from df_test_Adjust group by NIVEL_EDUCATIVO").display()

In [0]:
sqlContext.sql("select " +
              "NIVEL_EDUCATIVO, AVG(EDAD),count(*) as TOTAL " +
               "from " +
               "df_test_Adjust " +
               "group by " +
               "NIVEL_EDUCATIVO " +
               "order by TOTAL desc"
              ).display()

In [0]:
#Ejercicio:
#1. Elabore una consulta donde se muestre los clientes mayores de 60 años con menores probabilidade de default (los 5 primeros)

sqlContext.sql("select " +
               "ID " +
               "from " +
               "df_test_Adjust " +
               "where " +
               "EDAD > 60 " +
               "order by PD asc " +
               "limit 5"
              ).display()


In [0]:
#2. Elabore una consulta donde se indique la cantidad de solteros menores de 30 años que tienen más de 3 tarjetas de credito y tienen menor probabilidad de default (10 primeros)

sqlContext.sql("select * " +
               "from " +
               "df_test_Adjust " +
               "where " +
               "RTRIM(LTRIM(ESTADO_CIVIL))='Soltero' AND " +
               "EDAD <= 30 AND " +
               "CAST(SUBSTRING(RTRIM(LTRIM(NUMERO_ENTIDADES)),1,1) AS INT) > 3 " +
               "order by PD asc " +
               "limit 10"
              ).display()


#CAST(SUBSTRING(RTRIM(LTRIM(NUMERO_ENTIDADES)),1,1) AS INT) >3

#sqlContext.sql(" select  * from df_test_Adjust where RTRIM(LTRIM(ESTADO_CIVIL))='Soltero' AND EDAD<=30 AND CAST(SUBSTRING(RTRIM(LTRIM(NUMERO_ENTIDADES)),1,1) AS INT) >3 order by PD asc limit 10").display()


In [0]:
data_heroes = [
   ('Batman',3),
   ('Superman',4),
   ('Deadpool',1),
]

data_origen = [
   ('Humano',3),
   ('Criptoniano',5),
   ('Mutante',2),
]

heroes = spark.createDataFrame(data_heroes,['name','id'])
origen = spark.createDataFrame(data_origen,['race','id'])
heroes.show()
origen.show()

In [0]:
#Registro los dataframes que he creado como tablas
sqlContext.registerDataFrameAsTable(heroes,"heroes")
sqlContext.registerDataFrameAsTable(origen,"races")

In [0]:
sqlContext.sql("select * from heroes inner join races on heroes.id = races.id").display()

In [0]:
sqlContext.sql("select * from heroes full outer join races on heroes.id = races.id").display()

In [0]:
sqlContext.sql("select * from heroes left join races on heroes.id = races.id").display()
sqlContext.sql("select * from heroes right join races on heroes.id = races.id").display()

In [0]:
sqlContext.sql("select * from heroes union select * from races").display()
sqlContext.sql("select * from heroes intersect select * from races").display()

In [0]:
df_test_adjust.count()

In [0]:
df_test_adjust.groupby('SEXO').count().display()

In [0]:
import pyspark.sql.functions as f
df_test_adjust.groupby('SEXO').agg(f.mean('EDAD')).display()

In [0]:
#joins
heroes.join(origen, on ='id',how='inner').show()
heroes.join(origen, on ='id',how='outer').show()
heroes.join(origen, on ='id',how='left').show()
heroes.join(origen, on ='id',how='right').show()

In [0]:
#Filtros
df_test_adjust.groupby('SEXO').agg(f.mean('EDAD')).filter(f.col('SEXO').isin('M')).show()


###Taller 3

In [0]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext

path = "/FileStore/tables/rides.txt"

schema = StructType([\
                     StructField("id",IntegerType(),True), \
                     StructField("open_app_id",IntegerType(),True), \
                     StructField("start_time",TimestampType(),True), \
                     StructField("finish_time",TimestampType(),True), \
                     StructField("starting_lng",StringType(),True), \
                     StructField("starting_lat",StringType(),True), \
                     StructField("dest_lng",StringType(),True), \
                     StructField("dest_lat",StringType(),True), \
                     StructField("price",StringType(),True), \
                     StructField("user_id",IntegerType(),True), \
                     StructField("scooter_id",IntegerType(),True)
                  ])

df_taller3 = spark.read.option("header",True).option("encoding", "ISO-8859-1").schema(schema).csv(path,sep=",")

In [0]:
df_taller3.show()

In [0]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)
sqlContext.registerDataFrameAsTable(df_taller3,"df_taller3") 

In [0]:
sqlContext.sql("SELECT start_time, finish_time FROM df_taller3 order by finish_time asc").display()


In [0]:

sqlContext.sql("SELECT substring(start_time,6,2) as month, avg(datediff(minute,start_time, finish_time)) as time FROM df_taller3 group by month").display()



In [0]:
sqlContext.sql("SELECT id, datediff(minute,start_time, finish_time) as time, cast(REPLACE(price, ',', '.') AS double) as price_cast FROM df_taller3 order by price_cast desc limit 500").display(500)

#avg(datediff(minute,start_time, finish_time)) as useTime
#cast(REPLACE(price, ',', '.') AS double)

In [0]:
sqlContext.sql("SELECT scooter_id, avg(datediff(minute,start_time, finish_time)) as time, AVG(cast(REPLACE(price, ',', '.') AS double)) as cost FROM df_taller3 group by scooter_id ").display()


#avg(datediff(minute,start_time, finish_time)) as time,

In [0]:
sqlContext.sql("SELECT user_id, count(user_id) as total_services FROM df_taller3 group by user_id order by total_services desc").display()


In [0]:
sqlContext.sql("SELECT user_id, count(user_id) as total_services FROM df_taller3 group by user_id order by total_services asc limit 100").display(100)
